# 构建transformer流程学习记录

##embedding 前的输入转化，为什么文字能输入到模型

In [ ]:
from os import close

import numpy as np
import torch
import torch.nn as nn
from torch.ao.nn.quantized import LayerNorm
from torch.autograd import Variable
from torch.nn.functional import dropout

# 构建词汇表
vocab = {"<pad>": 0, "hello": 1, "world": 2, "this": 3, "is": 4, "a": 5, "test": 6}
vocab_size = len(vocab)  # 词汇表大小
embedding_dim = 5  # 嵌入向量的维度

# 创建嵌入层
embedding = nn.Embedding(vocab_size, embedding_dim)

# 将文本转换为数值索引
text = "hello world this is"
indices = [vocab[word] for word in text.split()]  # 数值化后的输入
x = torch.tensor(indices)  # 转换为张量

# 使用嵌入层
embedded_vectors = embedding(x)

print("输入文本:", text)
print("数值化后的输入:", x)
print("嵌入矩阵的形状:", embedding.weight.shape)
print("输出的嵌入向量:", embedded_vectors)

## 构建embedding 层
本质上embedding层是一个词汇表，将文字输入embedding层的作用就是将转化为索引后的汉字变为一个自定义维度的向量（常用的是512）

In [ ]:
import math
#embedding
import torch
from torch import nn
import torch.nn.functional as F

class Embedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Embedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        #这里是在创建一个vocab_size行embedding_dim列的嵌入层张量，也可以理解为一个索引为vocab_size,value为embedding_dim的字典。
        self.embedding_dim = embedding_dim
        #这里是为了在后面缩放输入，是的输入的数据在训练过程中稳定。
        #但是我差一个例子来说明，如果没有这个缩放过程会有是吗影响

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.ebedding_dim)
        #这里x是经过tokenizer转化后的索引（数字）,embedding(x)的作用就是将x的里面的索引数字和embedding层里的索引进行匹配然后返回一个embedding_dim维度的向量。简单理解就是embedding(x)的作用就是在查表。

## 位置编码
为每个向量加上位置信息

In [ ]:
#创建位置编码
class positional_encoding(nn.Module):
    def __init__(self,d_dim,dropout, max_len = 5000):
        super(positional_encoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position_encoding_matrix = np.zeros((max_len,d_dim))
        # 创建每个输入大小的位置编码
        position = torch.arange(0,d_dim).unsqueeze(dim = 0)
        # 创建一个(1，d_dim)的张量方便后续通过广播计算
        div_term = math.exp(- torch.arange(0,d_dim,2) * math.log(10000.0) / d_dim)
        # 公式的基础写法
        position_encoding_matrix[:, 0::2] = torch.sin(position * div_term)
        position_encoding_matrix[:, 1::2] = torch.cos(position * div_term)
        position_encoding_matrix = position_encoding_matrix.unsqueeze(dim = 0)
        # 这里unsqueeze的作用是为其增加一个batch_size的维度是的这个位置编码可以通过广播机制加载到每个批次的输入上
        self.register_buffer('position_encoding_matrix', position_encoding_matrix)
        # 这里使用register_buffer是为了保障其不会被优化器改变

    def forward(self, x):
        x = x + self.position_encoding_matrix[:,x.size(1)]
        # 这里的x.size是为x匹配对应长度的位置编码
        return self.dropout(x)

## 构建解码器

In [ ]:
class Generator(nn.Module):
    def __init__(self, vocab_size, d_dim):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_dim, vocab_size)
        # 主要作用是把上一层的输出的每个序列对应的高纬度特征变为与词表的对应数值分布。然后在下面的softmax操作中变为对应概率分布，方便后续输出结果。

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [ ]:
import copy

# 构建层复制器
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
# 构建编码器
# 既然解码器里有多个层，每个层又有两个子层，分别是自注意力层和前馈全连接层，每个子层后面跟着一个残差网络和归一化。为什么在encoder里没有体现？而只是简单的进行了最后一次的layernorm?
# 编码器里没有直接写编码层而是另外写了一个编码层类
class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = nn.LayerNorm(layer.size)
        # LayerNorm的作用和执行？

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x,mask)
        return self.norm(x)

In [ ]:
# 构建编码层
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.size = size
        # 设置这个size是干嘛的
        self.sublayer = clones(SublayerConnection((size,dropout), 2))
        # 这个sublayerconnection又自己实现，包含了 残差网络和归一化层在里面

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x:self.self_attn(x,x,x,mask))
        z = self.sublayer[1](x, self.feed_forward)
        # 为什么这里换成了Z?
        return z


In [ ]:
# 还差一个自注意力机制的实现代码